In [1]:
##DSC478
#Assignment 3 Question 2 Clustering
#Ke Feng 

In [1]:
#import libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.cluster import KMeans 
from sklearn.metrics import completeness_score, homogeneity_score
from numpy import *

In [2]:
###a. Create a Cos distance function
#x=instance D=data
def cosdist(x, D):
    norm_D = np.linalg.norm(D)
    norm_x = np.linalg.norm(x)
    similarity = np.dot(D, x)/(norm_D * norm_x)
    distance = 1 - similarity
    return distance

In [3]:
###b.Load the data (Transpose). Then split the data into 80-20 for clustering
#Load the data
matrix=pd.DataFrame(np.genfromtxt('matrix.txt', delimiter=',', dtype=int))
classes = pd.DataFrame(np.genfromtxt('classes.txt', delimiter=' ', dtype=int, skip_header=1))
terms = (np.genfromtxt ('terms.txt', delimiter='\t', dtype = str))

In [4]:
#Transpose Matrix doc
matrixT=matrix.T

In [5]:
#Convert Data into DF * IDF weights
numTerms=len(terms)
DF = np.array([(matrixT!=0).sum(0)])
NDocs=len(matrixT[0])
newmatrix=np.ones(np.shape(matrixT), dtype=float) * NDocs
DivM=np.divide(newmatrix, DF)
IDF=np.log2(DivM)
np.set_printoptions(precision=2, suppress=True)
#Compute the weights
matrix_tfidf=matrixT * IDF 
matrix_tfidf.head()

,0,1,2,3,4,5,6,7,8,9,...,9318,9319,9320,9321,9322,9323,9324,9325,9326,9327
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.861448,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#Split the dat into 80-20 training and testing 
xtrain, xtest, ctrain, ctest = train_test_split(matrix_tfidf, classes, test_size = 0.2, random_state = 33)

In [7]:
###c.Perform K-means clustering
#Write K-Means Function, Referencing Week 7 Class Notes kMeans
#Distance Function and Define Centroids
def distEuclid(vecA, vecB):
    return sqrt(sum(power(vecA - vecB, 2))) #la.norm(vecA-vecB)

def cosdist(x, D):  #Asked to use Cosine 
    norm_D = np.linalg.norm(D)
    norm_x = np.linalg.norm(x)
    similarity = np.dot(D, x)/(norm_D * norm_x)
    distance = 1 - similarity
    return distance

def randCent(dataSet, k):
    n = shape(dataSet)[1]
    centroids = zeros((k,n), dtype=float)
    for j in range(n): #create random cluster centers
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * random.rand(k)
    return centroids 

def kmeans(dataSet, k, distMeas=cosdist, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = zeros((m,2))
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):#for each data point assign it to the closest centroid
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        # print centroids
        for cent in range(k):#recalculate centroids
            ptsInClust = dataSet[nonzero(clusterAssment[:,0]==cent)[0]] #get all the point in this cluster - Note: this was incorrect in the original distribution.
            if(len(ptsInClust)!=0):
                centroids[cent,:] = mean(ptsInClust, axis=0) #assign centroid to mean - Note condition was added 10/28/2013
    return centroids, clusterAssment

In [8]:
#print out centroids and clusters to see if the functions are valid 
xtrain = np.array(xtrain)
centroids, clusterAssment = kmeans(xtrain, 5)
print ('\t\tCluster1\tCluster2\tCluster3\tCluster4\tCluster5')
for i in range(len(terms)):
    print ('%10s\t%.4f\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.4f' %(terms[i],centroids[0][i],centroids[1][i],centroids[2][i], centroids[3][i], centroids[4][i]))

		Cluster1	Cluster2	Cluster3	Cluster4	Cluster5
        aa	0.0000		0.0000		0.0605		0.0000		0.0414
     aargh	0.0000		0.0000		0.0000		0.0829		0.0000
     aaron	0.0000		0.0000		1.1236		0.0163		0.0622
    aaronc	0.0000		0.0000		0.1644		0.0000		0.0000
        ab	0.9484		0.0000		0.2497		0.0000		0.0395
   abandon	0.0000		0.0000		0.1233		0.0387		0.0211
       abc	0.0000		0.0313		0.0000		0.6381		0.0000
      abid	0.0000		0.0238		0.0246		0.0000		0.0000
      abil	0.0000		0.3089		0.2081		0.1044		0.0926
       abl	0.0000		0.4732		0.3715		0.3034		0.2912
    abomin	0.0000		0.0000		0.0941		0.0000		0.0000
     abort	0.0000		0.0238		0.0246		0.0000		0.0000
    abound	0.0000		0.0000		0.1135		0.0000		0.0000
   abraham	0.0000		0.0416		0.0859		0.0000		0.0000
    abridg	0.0000		0.0911		0.0000		0.0000		0.0000
    abroad	0.0000		0.0911		0.0000		0.0000		0.0000
    absenc	0.0000		0.0796		0.1233		0.0000		0.0000
    absent	0.0000		0.0624		0.0429		0.0000		0.0000
   absolut	0.0000		0.1070		0.8148		0.1299		0.0851
   

     damag	0.0000		0.1615		0.2168		0.0314		0.0685
      damn	0.0000		0.1868		0.0742		0.1396		0.0762
    damnat	0.0000		0.0000		0.0881		0.0207		0.0000
 damphouss	0.0000		0.0000		0.0000		0.1864		0.0000
       dan	0.0000		0.1300		0.3433		0.0561		0.0690
      dana	0.0000		0.0000		0.0246		0.0000		0.0505
      danc	0.0000		0.0179		0.0369		0.0174		0.0663
      dang	0.0000		0.0000		0.0000		0.0000		0.0679
    danger	0.0000		0.1599		0.3003		0.0282		0.0154
    daniel	0.0000		0.0160		0.0992		0.1089		0.1019
     danni	0.0000		0.1488		0.0000		0.0723		0.0296
      dare	0.0000		0.0000		0.0505		0.3638		0.0173
 daredevil	0.0000		0.0000		0.1474		0.0000		0.0000
     darin	0.0000		0.0000		0.1135		0.0000		0.0233
     dariu	0.0000		0.0000		0.2420		0.0000		0.0000
dariuslecoint	0.0000		0.0000		0.1210		0.0000		0.0000
      dark	0.0000		0.0630		0.3256		0.0459		0.0585
  darkhawk	0.0000		0.0000		0.1176		0.0000		0.0121
      darn	0.0000		0.0228		0.0000		0.0000		0.0362
    darren	0.0000		0.0000		0.0839		0.0789		0.01

      knee	0.0000		0.0199		0.0000		0.1546		0.0000
      knew	0.0000		0.0798		0.5773		0.0776		0.0494
    knickl	0.0000		0.0000		0.0000		0.1548		0.0000
     knife	0.0000		0.0220		0.0000		0.0640		0.0117
    knight	0.0000		0.0390		0.1210		0.0569		0.0104
     knock	0.0000		0.0390		0.0000		0.1517		0.0000
      know	0.0000		0.9199		1.5094		0.7183		0.5815
  knowledg	0.0000		0.2023		0.5354		0.1351		0.0402
   knowleg	0.0000		0.0879		0.0000		0.0000		0.0000
     known	0.0000		0.5005		0.4448		0.1357		0.0494
    koberg	0.0000		0.0000		0.3435		0.0000		0.0000
     kocur	0.0000		0.0000		0.0000		0.2079		0.0000
    koitai	0.0000		0.0000		0.0983		0.0000		0.0000
     koivu	0.0000		0.0000		0.0000		0.0924		0.0000
      kong	0.0000		0.0000		0.0246		0.0000		0.0252
konstantinov	0.0000		0.0000		0.0000		0.0640		0.0117
    koontz	0.0000		0.1099		0.0000		0.0000		0.0000
   koontzd	0.0000		0.1099		0.0000		0.0000		0.0000
     korea	0.0000		0.0220		0.0454		0.0000		0.0233
    korean	0.0000		0.0879		0.0227		0.0000		0.000

     ripem	0.0000		0.7291		0.0000		0.0000		0.0204
ripemmsuedu	0.0000		0.0911		0.0000		0.0000		0.0121
      risc	0.0000		0.0597		0.0000		0.0387		0.1900
      rise	0.0000		0.0320		0.3804		0.0000		0.0000
     risen	0.0000		0.0000		0.1469		0.0197		0.0000
      risk	0.0000		0.2173		0.1283		0.0302		0.0494
       rit	0.0000		0.0000		0.0000		0.0231		0.0252
      rite	0.0000		0.0000		0.2576		0.0000		0.0000
    ritual	0.0000		0.0000		0.3083		0.0000		0.0000
ritvaxiscritedu	0.0000		0.0000		0.0000		0.0000		0.0505
   rivalri	0.0000		0.0000		0.0000		0.1106		0.0000
     river	0.0000		0.0000		0.0220		0.0207		0.0340
    rivest	0.0000		0.1138		0.0000		0.0000		0.0000
        rj	0.9957		0.0586		0.0807		0.0000		0.0104
        rk	0.0000		0.0238		0.4422		0.0000		0.0000
        rl	1.0600		0.0208		0.2147		0.0000		0.0000
   rlglend	0.0000		0.2844		0.0000		0.0000		0.0000
       rlh	0.0000		0.0000		0.0983		0.0000		0.0000
       rlk	2.2414		0.0000		0.3405		0.0000		0.0000
        rm	1.5006		0.1324		0.3039		0.2001		0

In [9]:
#Referencing DePaul's Professor Mombasher's Notes, Output 
def printresults(data, K, numterms):
    centroids, clusterAssment = kmeans(mat(data), K, distMeas=cosdist)
    for i in range(K):
        print('Cluster', i+1)
        clusterm=data[clusterAssment[:, 0] == i]
        print ('Number of items in Cluster', i+1, ':', clusterm.shape[0])
        itemsnum=np.array([(clusterm.T !=0).sum(1)]).T
        mc = itemsnum / float(clusterm.shape[0])
        mc = map(list, mc)
        mc = [m[0] for m in mc]
        itemsnum = [m[0] for m in itemsnum]
        n = sorted(zip(terms, itemsnum, mc),key = lambda x:x[2], reverse = True)
        for m in n[:numterms]:
            print('Term: ',m[0])
            print('Doc Frequency: ',m[1])
            print('Percent of docs include the term ', m[0], ': ', m[2] * 100,'\n')
        print('\n')
    return centroids, clusterAssment

In [10]:
#Print top n terms, cluster DF value for each term, and size of the cluster 
printresults(xtrain, 5, 10)

Cluster 1
Number of items in Cluster 1 : 475
Term:  sale
Doc Frequency:  225
Percent of docs include the term  sale :  47.368421052631575 

Term:  email
Doc Frequency:  151
Percent of docs include the term  email :  31.789473684210527 

Term:  pleas
Doc Frequency:  125
Percent of docs include the term  pleas :  26.31578947368421 

Term:  game
Doc Frequency:  108
Percent of docs include the term  game :  22.736842105263158 

Term:  offer
Doc Frequency:  107
Percent of docs include the term  offer :  22.526315789473685 

Term:  on
Doc Frequency:  106
Percent of docs include the term  on :  22.315789473684212 

Term:  thank
Doc Frequency:  102
Percent of docs include the term  thank :  21.47368421052632 

Term:  includ
Doc Frequency:  96
Percent of docs include the term  includ :  20.210526315789473 

Term:  ship
Doc Frequency:  96
Percent of docs include the term  ship :  20.210526315789473 

Term:  get
Doc Frequency:  92
Percent of docs include the term  get :  19.36842105263158 



Clu

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


(array([[  0.03,   0.05,   0.06, ...,   0.  ,   0.  ,   0.  ],
        [  7.95,   7.9 , 108.49, ...,   4.17,   1.75,   5.46],
        [  0.  ,   0.  ,   0.  , ...,   1.39,   0.  ,   0.  ],
        [  0.04,   0.  ,   0.05, ...,   0.  ,   0.  ,   0.  ],
        [  0.02,   0.01,   0.41, ...,   0.02,   0.01,   0.05]]),
 array([[0.  , 0.81],
        [0.  , 0.9 ],
        [0.  , 0.77],
        ...,
        [4.  , 0.48],
        [3.  , 0.59],
        [4.  , 0.78]]))

In [11]:
###d. Calculate completeness and Homogeneity values 
ctrainT=np.array(ctrain.T)
cscore=completeness_score(ctrainT[0], clusterAssment[:, 0])
hscore=homogeneity_score(ctrainT[0], clusterAssment[:, 0])
print('The Completeness Score is %s' % cscore)
print('The Henogeneity Score is %s' % hscore)

The Completeness Score is 1.0
The Henogeneity Score is 0.179184681205318


In [12]:
###e. Categorizing this using Cosine Similarity
results = []
for d in xtest:  
    dic = {}
    sim = []
    c = 0
    for centroid in centroids:
        c += 1
        similarity=cosdist(d, centroid)
        sim.append(similarity.any())
        dic['Cosine Similarity' + str(c)] = similarity
    dic['Pred Class'] = sim.index(max(sim)) +1
    results.append(dic)
test = pd.DataFrame(results)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [13]:
test.head()

,Cosine Similarity1,Cosine Similarity2,Cosine Similarity3,Cosine Similarity4,Cosine Similarity5,Pred Class
0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",1
1,"[1.0, 1.0, 1.0, 1.0, 0.9999774590420912, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9984340472071...","[0.9978975779860074, 1.0, 0.9609523568565672, ...","[1.0, 0.9954340750953515, 0.9991043802931178, ...","[0.9960611240515805, 1.0, 0.9940912831279528, ...",1
2,"[1.0, 1.0, 1.0, 1.0, 0.9999774590420912, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9984340472071...","[0.9978975779860074, 1.0, 0.9609523568565672, ...","[1.0, 0.9954340750953515, 0.9991043802931178, ...","[0.9960611240515805, 1.0, 0.9940912831279528, ...",1
3,"[1.0, 1.0, 1.0, 1.0, 0.9999774590420912, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9984340472071...","[0.9978975779860074, 1.0, 0.9609523568565672, ...","[1.0, 0.9954340750953515, 0.9991043802931178, ...","[0.9960611240515805, 1.0, 0.9940912831279528, ...",1
4,"[1.0, 1.0, 1.0, 1.0, 0.9999774590420912, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9984340472071...","[0.9978975779860074, 1.0, 0.9609523568565672, ...","[1.0, 0.9954340750953515, 0.9991043802931178, ...","[0.9960611240515805, 1.0, 0.9940912831279528, ...",1
